In [1]:
import boto3

from boto3.dynamodb.conditions import Key, Attr

import pandas as pd

from pandasgui import show

from IPython.display import clear_output

In [2]:
dynamodb = boto3.resource( 'dynamodb' )

In [3]:
adspp_table = dynamodb.Table( 'ads_passenger_processed' )

In [4]:
query_input = dict(
                   IndexName = 'topic-index',
                   # Select =
                   Limit = 1000,
                   # ConsistentRead =
                   # ScanIndexForward =
                   # ExclusiveStartKey =
                   # ReturnConsumedCapacity =
                   # ProjectionExpression =
                   # FilterExpression =
                   KeyConditionExpression = Key( 'topic' ).eq( '/apollo/sensor/gnss/best_pose' )
                   # ExpressionAttributeNames =
                   # ExpressionAttributeValues =
                  )

query_output = adspp_table.query( **query_input )

query_output_df = pd.DataFrame.from_dict( pd.json_normalize( query_output[ 'Items' ] ), orient = 'columns' )

In [8]:
dict.keys( query_output )

print( query_output[ 'Items'][ 0 ] )

{'topic': '/apollo/sensor/gnss/best_pose', 'gpsGlonassUsedMask': Decimal('51'), 'time': Decimal('1696267162048124370'), 'longitudeStdDev': Decimal('0.012680259'), 'undulation': Decimal('-35'), 'numSatsTracked': Decimal('17'), 'baseStationId': 'MzkzNA==', 'latitudeStdDev': Decimal('0.018686043'), 'header': {'moduleName': 'gnss', 'sequenceNum': Decimal('1992'), 'timestampSec': Decimal('1696267162.048053')}, 'differentialAge': Decimal('1'), 'solType': 'NARROW_INT', 'galileoBeidouUsedMask': Decimal('0'), '_id': '7e9fb9a9-a553-11ee-88ec-eb6a8d5269b4', 'latitude': Decimal('39.3328270159293'), 'numSatsL1': Decimal('15'), 'datumId': 'WGS84', 'msgsize': Decimal('119'), 'measurementTime': Decimal('1380302380'), 'extendedSolutionStatus': Decimal('33'), 'metadataID': '6e6dab62-a553-11ee-88ec-eb6a8d5269b4', 'heightStdDev': Decimal('0.04997633'), 'numSatsMulti': Decimal('14'), 'heightMsl': Decimal('207.59663231670856'), 'groupMetadataID': '2bc6ebb8-a529-11ee-88ec-eb6a8d5269b4', 'longitude': Decimal(

In [16]:
while ( 'LastEvaluatedKey' in query_output.keys() ):

    print( query_output_df.shape )

    query_input[ 'ExclusiveStartKey' ] = query_output[ 'LastEvaluatedKey' ]

    query_output = adspp_table.query( **query_input )

    temp_df = pd.DataFrame.from_dict( pd.json_normalize( query_output[ 'Items' ] ), orient = 'columns' )

    query_output_df = pd.concat( [ query_output_df, temp_df ] )

    clear_output( wait = True )

(446000, 31)


In [36]:
groupMetadataID_col = query_output_df['groupMetadataID'].tolist()

In [37]:
unique_groupMetadataIDs = list( set( groupMetadataID_col ) )

In [38]:
data = { 'groupMetadataID' : unique_groupMetadataIDs }

unique_groupMetadataIDs_df = pd.DataFrame( data )

In [39]:
print( unique_groupMetadataIDs_df )

                          groupMetadataID
0    41cd65b4-f0aa-11ee-b9e8-fb353e7798cd
1    2d35c522-eba2-11ee-b297-3b0ad9d5d6c6
2    5c7a9ab2-f13b-11ee-ba72-fb353e7798cd
3    f8fd0fd8-f243-11ee-bb3f-fb353e7798cd
4    4ed017ee-ef05-11ee-9385-ef789ffde1d3
..                                    ...
325  25e27b86-f06a-11ee-b9a3-fb353e7798cd
326  60546ef4-edaa-11ee-9385-ef789ffde1d3
327  fe973c9c-f53c-11ee-8afa-cb629b0d53e6
328  5240e750-ec30-11ee-b297-3b0ad9d5d6c6
329  c338788a-d324-11ee-b437-336917683bb8

[330 rows x 1 columns]


In [30]:
unique_groupMetadataIDs_df.to_csv( r'/home/rcoldren_linux/Desktop/Data/groupMetadataID_list.csv', index = False )

In [31]:
query_output_df.to_csv( r'/home/rcoldren_linux/Desktop/Data/apollo_sensor_gnss_best_pose_Data/apollo_sensor_gnss_best_pose_all_gmdIDs.csv' )